# 13 - Prueba de Concepto: Tendencias Políticas en Twitter


* Vamos a realizar una prueba de concepto que consiste en:
    1. Entrenar un modelo
    2. Exportar el modelo
    3. Explotar el modelo
    

* En esta PoC vamos a tener como ***dataset*** un conjunto de ***tweets etiquetados con el nombre de un partido político***.


* Con estos ***tweets***; una vez ***Normalizados*** y creada la ***bolsa de palabras***, vamos a ***crear y evaluar*** una serie de ***modelos creados con Algoritmos de Aprendizaje de Clasificación***.


* Una vez evaluados ***nos quedaremos con el Algoritmos de Aprendizaje que mejor modelo genere*** y ***crearemos un modelo con el Algoritmos de Aprendizaje seleccionado entrenandolo con todo el dataset***. Posteriormente ***exportaremos ese modelo*** para su explotación.


* Por último '*simulando un producto*' ***leeremos los tweets de una determinda cuenta de twitter*** y ***pasandole los tweets al modelo*** (previamente importado), nos ***clasificará esos tweets según su tendencia política***.


* Para realizar todo esto y diferenciar lo que es la ***generación de modelos*** y ***explotación de modelos*** vamos a realizar todo este proceso en dos notebooks.


# 1. Generación de modelos

* Notebook: *13_PoC_Tendencias_Politicas_Twitter_Generacion_Exportacion_Modelos.ipynb*


* En este punto realizaremos las siguientes acciones:

    1. Carga de datos (tweets)
    2. Normalización de los tweets
    3. Creacción de la Bolsa de Palabras (BoW)
    4. Particionado de los datos con el método del 'Cross Validation'
    5. Creacción de modelos y evaluación
    6. Elección del mejor modelo y exportación en pickle
        * Exportar el modelo creado por un algoritmo de aprendizaje
        * Exportar la bolsa de palabras


# 2. Explotación de modelos

* Notebook: *14_PoC_Tendencias_Politicas_Twitter_Prediccion.ipynb*


* En este punto realizaremos las siguientes acciones:

    1. Lectura (via API) de los tweets de una determinada cuenta de twitter
    2. Normalización de los tweets
    3. Importación de los modelos (Clasificación y BoW)
    3. Creacción de la Bolsa de Palabras (BoW) de los nuevos tweets
    4. Predicción

<hr>


## Carga de Datos


* El primer paso que vamos a realizar es el de cargar los datos. 


* Este fichero lo podemos leer como un '*csv*' con pandas pasandole como separador '***::::***'.


* Este fichero esta estructurado de la siguiente manera
    - **Cuenta**: Cuenta de twitter
    - **Partido**: Partido político al que pertenece (ciudadanos, podemos, pp, psoe)
    - **Timestamp**: Instante en el que se publicó el tweet
    - **Tweet**: Tweet.
    
    
* Leemos los datos y mostramos una muestra:

In [1]:
import pandas as pd

tweets_file = './data/tweets_politica.csv'
df = pd.read_csv(tweets_file, sep='::::', engine='python')
print('Número de Tweets Cargados: {num}'.format(num=df.shape[0]))
df.sample(10)

Número de Tweets Cargados: 3843


,cuenta,partido,timestamp,tweet
1664,sanchezcastejon,psoe,1557858287,"En #Santiago, con el candidato a la alcaldía, ..."
2837,ahorapodemos,podemos,1558629554,"🎥 ""Nos quieren esconder dónde se deciden las c..."
131,PSOE,psoe,1556785931,✊🏽🌹 @abalosmeco Cumplimos 140 años porque somo...
613,sanchezcastejon,psoe,1555563308,Acabo de terminar mis 10 km diarios de #runnin...
2366,PSOE,psoe,1558177124,¡S E G U I M O S! No paramos en este sábado de...
2183,PSOE,psoe,1558086146,🔴 @abalosmeco Seguiremos trabajando por nuestr...
2106,TeoGarciaEgea,pp,1557950615,¿Harás que pase otra vez el 26 de mayo? #VotaP...
3061,InesArrimadas,ciudadanos,1558617960,La Sra. Batet empieza su andadura al frente de...
1966,PSOE,psoe,1557940875,"📺@TimmermansEU en @24h_tve: Para Europa, l@s s..."
1083,carmencalvo_,psoe,1556214089,En encuentro @psoetorrijos @pscmpsoe: Ciudadan...


* Para esta PoC vamos a utilizar solo el '***Tweet***' y el '***Partido Político***' asociado al tweet.

In [2]:
tweets = [tuple(x) for x in df[['tweet', 'partido']].values]
print('Número de Tweets Cargados: {num}'.format(num=len(tweets)))

Número de Tweets Cargados: 3843


<hr>


## Normalización

* Utilizamos ***spaCy*** para la tokenización y normalización.


* Tras realizar un análisis del contenido de los tweets pasamos a realizar las siguientes acciones para ***normalizar*** los tweets:
    1. Pasamos las frases a minúsculas.
    2. Sustituimos los puntos por espacios ya que hay muchas palabras unidas por un punto
    3. Quitamos la almuhadilla de los hashtags para considerarlos como palabras.
    4. Eliminamos los signos de puntuación.
    5. Eliminamos las palabras con menos de 3 caracteres.
    6. Eliminamos las Stop-Words.
    7. Eliminamos los enlaces(http) y las menciones (@)
    8. Pasamos la palabra a su lema


* Todos estos pasos los vamos a realizar en una misma función.


* ***NOTA***: Se pueden realizar más acciones de normalización que las realizadas, como tratamiento de emoticonos, tratamiento especial de referencia a cuentas, hashtags, etc. Al tratarse de un PoC didáctica se ha realizado una normalización '*sencilla*'.

In [3]:
import numpy as np
import spacy
nlp = spacy.load('es_core_news_sm')

# Divido los datos en dos listas 
#     X: los tweets
#     y: target (polaridad)

X = [doc[0] for doc in tweets]
y = np.array([doc[1] for doc in tweets])

def normalize(sentenses):
    """normalizamos la lista de frases y devolvemos la misma lista de frases normalizada"""
    for index, sentense in enumerate(sentenses):
        # Tokenizamos el tweets realizando los puntos 1,2 y 3.
        sentense = nlp(sentense.lower().replace('.', ' ').replace('#', ' '))
        # Puntos 4,5,6,7 y 8
        sentenses[index] = " ".join([word.lemma_ for word in sentense if (not word.is_punct)
                                     and (len(word.text) > 2) and (not word.is_stop) 
                                     and (not word.text.startswith('@')) 
                                     and (not word.text.startswith('http'))
                                     and (not ':' in word.text)])
    return sentenses

# Normalizamos las frases
X_norm = normalize(X)

<hr>


## Bolsa de palabras

* Pasamos a construir una bolsa de palabras de frecuencias.


* Vamos a utilizar (para construir la bolsa de palabras) la clase "*CountVectorizer*" de scikit, quedandonos con:
    - Las 1000 palabras más frecuentes.
    - Que las palabras aparezcan por los menos en 5 tweets.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=1000, min_df=5)
vectorizer.fit_transform(X_norm)

# Pasamos los tweets normalizados a Bolsa de palabras
X_bow = vectorizer.transform(X_norm)

<hr>


## Particionado de Datos (Cross Validation)

* En este ejercicio el objetivo es ver cual es el mejor modelo que podemos construir para clasificar tweets según la tendencia política, por tanto utilizaremos el método de evaluación del ***Cross Validation*** para que todos los tweets sean (en algún momento) de entrenamiento y test.


* Para ello dividiremos el conjunto de tweets en ***10 conjuntos*** para que en cada iteraccion uno de esos grupos sea el conjunto de test y el resto sea el conjunto de entrenamiento.


* Para realizar este particionamiento de datos utilizaremos la clase ***KFold*** de Scikit-Learn: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

In [5]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state=None, shuffle=True)

<hr>


## Creacción de Modelos y Evaluación (Accuracy)


* Vamos a crear y evaluar una serie de modelos para ver cual es que obtiene mejores resultados.


* Los modelos que vamos a crear y evaluar son los siguientes:
    - Multinomial Naive Bayes
    - Bernoulli Naive Bayes
    - Regresion Logistica
    - Support Vector Machine
    - Random Forest
    

* Para simplificar el problema solo vamos a evaluar los modelos (***Cross Validation***) con el accuracy y nos quedaremos con el modelo que mejor accuracy tenga.


In [6]:
import warnings
warnings.filterwarnings("ignore")
import statistics
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

mnb = MultinomialNB()
bnb = BernoulliNB()
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000)
svm = LinearSVC()
rf = RandomForestClassifier(max_depth=50, n_estimators=50, max_features=5)

clasificadores = {'Multinomial Naive Bayes': mnb,
                  'Bernoulli Naive Bayes': bnb,
                  'Regresion Logistica': lr,
                  'Support Vector Machine': svm,
                  'Random Forest': rf}

# Ajustamos los modelos y calculamos el accuracy para los datos de entrenamiento
accuracy = list()
for k, v in clasificadores.items():
    print ('CREANDO MODELO: {clas}'.format(clas=k))
    model = {}
    model['name'] = k
    model['acc_list'] = list()
    for train_index, test_index in kf.split(X_bow):
        v.fit(X_bow[train_index], y[train_index])
        acc = v.score(X_bow[test_index], y[test_index])
        model['acc_list'].append(acc)
    model['acc'] = statistics.mean(model['acc_list'])
    accuracy.append(model)
    print ('\tAVG Accuracy: {avg:0.4f}'.format(avg=statistics.mean(model['acc_list'])))

CREANDO MODELO: Multinomial Naive Bayes
	AVG Accuracy: 0.7718
CREANDO MODELO: Bernoulli Naive Bayes
	AVG Accuracy: 0.7996
CREANDO MODELO: Regresion Logistica
	AVG Accuracy: 0.7973
CREANDO MODELO: Support Vector Machine
	AVG Accuracy: 0.7804
CREANDO MODELO: Random Forest
	AVG Accuracy: 0.8020


In [7]:
# Pasamos los resultados a un DataFrame para visualizarlos mejor
results = pd.DataFrame.from_dict(accuracy)
results.set_index("name", inplace=True)
results.head()

,acc,acc_list
name,,
Multinomial Naive Bayes,0.771803,"[0.7844155844155845, 0.7610389610389611, 0.732..."
Bernoulli Naive Bayes,0.799617,"[0.8311688311688312, 0.8467532467532467, 0.792..."
Regresion Logistica,0.797282,"[0.8441558441558441, 0.8025974025974026, 0.789..."
Support Vector Machine,0.780373,"[0.7896103896103897, 0.8155844155844156, 0.763..."
Random Forest,0.801977,"[0.7896103896103897, 0.8311688311688312, 0.787..."


<hr>


# Elección del mejor modelo y exportación en ***Pickle***


* Parace que el mejor modelo (a nivel de accuracy) es el creado por el algoritmo de aprendizaje de ***Bernoulli Naive Bayes***.



## 1. Creacción del Modelo con todos los tweets


* Una vez que hemos seleccionado el Algoritmo de Aprendizaje (***Bernoulli Naive Bayes***) que mejor modelos genera, vamos a crear un modelo utilizando todos los datos (tweets) disponibles en el dataset.


* Pasamos a crear el modelo con todos los datos:

In [8]:
from sklearn.naive_bayes import BernoulliNB

model_bnb = BernoulliNB()
model_bnb.fit(X_bow, y)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

## 2. Exportación del Modelo Pickle


* Ahora vamos a exportar el modelo creado por el Algoritmo de Aprendizaje ***Bernoulli Naive Bayes*** en formato pickle.


* El modelo creado se ha generado con una determinada ***bolsa de palabras*** que contenia 1000 Palabras en su diccionario. Por tanto tambien tenemos que exportar el modelo generado por el ***CountVectorizer*** para que los nuevos tweets a predecir se ajusten a esa BoW.


* Por tanto tenemos que realizar:
    1. Exportar el ***Modelo*** generado por el Algoritmo de Aprendizaje ***Bernoulli Naive Bayes***.
    2. Exportar la ***Bolsa de Palabras*** generado por la clase ***CountVectorizer***.
    
    
* Exportamos el modelo generado por el Algoritmo de Aprendizaje ***Bernoulli Naive Bayes***.

In [9]:
import pickle

filename = './models/bernoullinb_tweets_politica.pickle'
save_model = open(filename,"wb")
pickle.dump(model_bnb, save_model) # con la función 'dump' guardamos el modelo
save_model.close()

* Exportar la ***Bolsa de Palabras*** generado por la clase ***CountVectorizer***.

In [10]:
filename = './models/vectorizer_bow_tweets_politica.pickle'
save_bow = open(filename,"wb")
pickle.dump(vectorizer, save_bow)
save_bow.close()

<hr>


# Bonus Track - Técnicas de Evaluación -

* Para evaluar las hipótesis obtenidas tras la aplicación de alguna de las técnicas de ML, es necesario disponer de un conjunto de datos (etiquetados o no) para generar la mejor de las hipótesis posibles y minimizar el error empírico


* Dado un conjunto de datos, podemos enumerar los siguientes métodos de evaluación en función de cómo se dividen los datos de entrenamiento y de test:

    - ***Resustitución***: Todos los datos disponibles se utilizan como datos de test y de entrenamiento.

    - ***Partición (Hold Out)***: Divide los datos en dos subconjuntos: uno de entrenamiento y uno de test 

    - ***Validación cruzada (Cross Validation)***: Divide los datos aleatoriamente en ‘N’ bloques. Cada bloque se utiliza como test para un sistema entrenado por el resto de bloques 

    - ***Exclusión individual (Leaving one out)***: Este método utiliza cada dato individual como dato único de test de un sistema entrenado con todos los datos excepto el de test 